# TODO: make instances instead of static classes so as to hold last forwarded inputs and outputs

In [3]:
from abc import ABC, abstractstaticmethod
import numpy as np

In [98]:
class CGNode(ABC):
    
    '''
    Computational graph node template. In all of the methods,
    the input vectors must have a shape of the form (n,)
    '''
    
    @abstractstaticmethod
    def prop_forward():
        
        '''
        Take inputs and evaluate the
        node's corresponding operation
        '''
        
        pass
    
    @abstractstaticmethod
    def prop_backward():
        
        '''
        Take outputs and evaluate the gradient
        of the node's corresponding operation
        '''
        
        pass

In [99]:
class CGSum(CGNode):
    
    '''
    Computational graph sum node
    '''
    
    @staticmethod
    def prop_forward(x):
        return np.sum(x)
    
    @staticmethod
    def prop_backward(x, grad):
        return grad * np.ones(np.array(x).size)


In [6]:
class CGMul(CGNode):
    
    '''
    Computational graph multiplication node
    '''
    
    @staticmethod
    def prop_forward(x):
        return np.prod(x)
    
    @staticmethod
    def prop_backward(x, grad):
        _x = np.array(x)
        
        # repeat x size times and arrange as rows
        repeated = np.ones([_x.size, _x.size])*x
        
        # start differentiation
        np.fill_diagonal(repeated, 1)
        
        # multiply constants 
        # (variables not being differentiated)
        return grad * np.prod(repeated, 1)

In [7]:
class CGExp(CGNode):
    
    @staticmethod
    def prop_forward(x):
        return np.exp(x)
    
    @staticmethod
    def prop_backward(x, grad):
        return grad * np.exp(x)

In [94]:
class CGDot(CGNode):
    
    '''
    Dot product node composed of summation and multiplication nodes for 
    transparency and accordance with the concept of a computational graph
    '''
    
    @staticmethod
    def prop_forward(x, y):
        
        xy = CGDot.pre_dot(x, y)
        mul = np.apply_along_axis(CGMul.prop_forward, 0, xy)
        result =  CGSum.prop_forward(mul)
        return result
    
    @staticmethod
    def prop_backward(x, y, grad):
        
        xy = CGDot.pre_dot()
        
        # take the inputs of the sum phase (i.e. the output of the multiplication phase)
        sum_back_prop = np.apply_along_axis(lambda _xy: CGSum.prop_backward(xy, grad), 0, xy)
        
        return s
    
    
    @staticmethod
    def pre_dot(x, y):
        
        '''
        Arrange the inputs to facilitate 
        dot product computation
        '''
        
        x = np.reshape(x, (1, len(x)))
        y = np.reshape(y, (1, len(y)))
        xy = np.concatenate((x,y))
        return xy

### I wanted to create a generalization of a computational graph, but its implementation appears to be too complex for my desired scope

In [9]:
# class CG:
    
#     def __init__(self, gates):
        
#         # gates is a list of lists of gates
#         # gates in the same row receive same inputs
#         self.gates = gates
        
#     def prop_forward(self, x):
#         next_values = x
#         for gate_row in self.gates:
#             input_values = next_values
#             next_values = []
#             for gate in gate_row:
#                 new_value = gate.prop_forward(input_values)
#                 next_values.append(new_value)
                
                